In [1]:
import numpy as np
import pandas as pd
import refesh_data as db

from datetime import datetime

In [2]:
def getNumCrowd():
    return db.dbNumCrowd(2023)

db_store = getNumCrowd()
db_store.head(5)

,recordtime,in_num,out_num,position,storeid
0,2023-02-15 09:49:00,18,5,Parking,36
1,2023-02-15 09:54:00,17,7,Parking,36
2,2023-02-15 09:59:00,12,5,Parking,36
3,2023-02-15 10:04:00,9,5,Parking,36
4,2023-02-15 10:09:00,15,4,Parking,36


In [ ]:
# Daily:   [y = None, d = '2023-03-10', w = None                 , m = None   , q = None]
# Weekly:  [y = 2023, d = None        , w = 'WK1 (01/01 - 01/01)', m = None   , q = None]
# Monthly: [y = 2023, d = None        , w = None                 , m = 'April', q = None]
# Quarter: [y = 2023, d = None        , w = None                 , m = None   , q = 0   ]
# Yearly:  [y = 2023, d = None        , w = None                 , m = None   , q = None]

# 2023-04-20 - <class 'datetime.date'>
# WK1 (01/01 - 01/01) - <class 'str'>
# April - <class 'str'>
# 0 - <class 'int'>
# 2023 - <class 'int'>

In [ ]:
def filter_data(data, store = 0, d = None, w = None, m = None, q = None, y = None):
    if (not y) and d:
        # print('Daily')
        data = data[data.recordtime.dt.date == d]
    else:
        data = data[data.recordtime.dt.year == y]
        if w:
            print('Weekly')
            data = data[data.recordtime.dt.date >= w[0] & data.recordtime.dt.date <= w[1]]
        elif m:
            # print('Monthly')
            data = data[data.recordtime.dt.strftime('%B') == m]
        elif q:
            # print('Quarter')
            data = data[data.recordtime.dt.to_period('Q').dt.strftime('%q').astype(int) == q + 1]

    if store > 0:
        data = data[data.storeid == store]

    return data

In [ ]:
data = db_store.copy()

d = datetime.strptime('2023-03-17', '%Y-%m-%d').date()
df_daily = filter_data(data, 'All', d = d)
df_daily

# df_weekly = filter_data(data, 'All', w = 'WK1 (01/01 - 01/01)', y = 2023)
# df_weekly

# df_monthly = filter_data(data, 'All', m = 'April', y = 2023)
# df_monthly

# df_quarter = filter_data(data, 'All', q = 4, y = 2022)
# df_quarter

# df_yearly = filter_data(data, 'All', y = 2021)
# df_yearly

In [ ]:
df = db_store.tail(205600).copy()
df['w'] = df['recordtime'].dt.strftime('%V').astype(int)
# df['s'] = df['w'].shift(1)
# df['week_num'] = np.where(df['s'].isna(), 1, df['w'] + 1)
# group = df.groupby('week_num').agg({'recordtime': ['min', 'max']}).reset_index()
# group['week'] = 'WK' + group['week_num'].astype(str) + \
#                 ' (' + group['recordtime']['min'].dt.strftime('%d/%m') + \
#                 ' - ' + group['recordtime']['max'].dt.strftime('%d/%m') + ')'

# df['week'] = df['recordtime'].dt.isocalendar().week

df.head()

In [ ]:
def getWeekNums(year):
    start_date = '1/1/' + year
    end_date = '31/12/' + year
    data = pd.date_range(start = start_date, end = end_date, freq = 'D')

    df = pd.DataFrame(data, columns = ['date'])

    df['w'] = df['date'].dt.strftime('%V').astype(int)
    df['s'] = df['w'].shift(1)
    df['week_num'] = np.where(df['s'].isna(), 1, df['w'] + 1)
    group = df.groupby('week_num').agg({'date': ['min', 'max']}).reset_index()
    group['week'] = 'WK' + group['week_num'].astype(str) + \
                    ' (' + group['date']['min'].dt.strftime('%d/%m') + \
                    ' - ' + group['date']['max'].dt.strftime('%d/%m') + ')'

    return group

df = getWeekNums('2023')
df.head(2)

In [ ]:
w = 7
a = (df.loc(0)[w][1], df.loc(0)[w][2])
a